<a href="https://colab.research.google.com/github/zeton24/gsn_iot_anomalies_detection/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Convolutional (Conv2d)
https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html#torch.nn.Conv2d

Average Pooling https://pytorch.org/docs/stable/generated/torch.nn.AvgPool2d.html#torch.nn.AvgPool2d

Normalization
https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm1d.html#torch.nn.BatchNorm1d

Spatial Dropout
https://pytorch.org/docs/stable/generated/torch.nn.Dropout2d.html#torch.nn.Dropout2d

Fully connected layer -> nn.Linear()


In [672]:
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim

In [673]:
import torch

## Elementy do wykorzystania

### Modele sieci 1D, 2D, 3D

In [674]:
class Model1D(nn.Module):
    def __init__(self):
        super().__init__()
        self.convblock1 = nn.Sequential(
            nn.Conv1d(in_channels=1, out_channels=32, kernel_size=5, padding='same'),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(32),
            nn.AvgPool1d(2),
            nn.Dropout(p=0.05)
        )

        self.convblock2 = nn.Sequential(
            nn.Conv1d(in_channels=32, out_channels=64, kernel_size=5, padding='same'),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(64),
            nn.AvgPool1d(2),
            nn.Dropout(p=0.05)
        )

        self.convblock3 = nn.Sequential(
            nn.Conv1d(in_channels=64, out_channels=128, kernel_size=5, padding='same'),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(128),
            nn.AvgPool1d(2),
            nn.Dropout(p=0.05)
        )

        self.convblock4 = nn.Sequential(
            nn.Conv1d(in_channels=128, out_channels=256, kernel_size=5, padding='same'),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(256),
            nn.AvgPool1d(2),
            nn.Dropout(p=0.05)
        )

        # Nazwa bloku do rozważenia, taka mi się wymyśliła, ale nie upieram się przy niej.
        self.evaluator = nn.Sequential(
            nn.Flatten(),
            nn.Linear(1024, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, 16)
            # Bez aktywacji na końcu, bo softmax się doda automatycznie razem z cross entropy.
        )

    def forward(self, x):
        x = self.convblock1(x)
        x = self.convblock2(x)
        x = self.convblock3(x)
        x = self.convblock4(x)
        x = self.evaluator(x)
        return x

In [675]:
class Model2D(nn.Module):
    def __init__(self):
        super().__init__()
        self.convblock1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=5, padding='same'),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(32),
            nn.AvgPool2d(2),
            nn.Dropout(p=0.05)
        )

        self.convblock2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, padding='same'),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(64),
            nn.AvgPool2d(2),
            nn.Dropout(p=0.05)
        )

        self.convblock3 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=5, padding='same'),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(128),
            nn.AvgPool2d(2),
            nn.Dropout(p=0.05)
        )

        self.convblock4 = nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=5, padding='same'),
            nn.ReLU(inplace=True),

            # Oni w artykule dają normalizację, ale pytorch wywala błąd.
            # Moim zdaniem słusznie, bo o ile rozumiem ten wzór na normalizację (co jest w dokumentacji),
            # to przy wymiarze wejścia [batch_size, 256, 1, 1] wyjście to tensor zer o takim samym wymiarze.
            # Po prostu normalizacja zmienia nam średnią na 0 (odchylenie standardowe też, ale to nieistotne),
            # a jak mamy 1 element, to zmiana średniej na 0, to zmiana elementu na 0.
            # Więc nawet jak jakoś obejdziemy ten błąd, to wyniki będą bez sensu.
            # Moim zdaniem w artykule jest błąd (a przynajmniej na rysunku, może w implementacji tego nie ma).

            # nn.BatchNorm2d(256),
            nn.Dropout(p=0.05)
        )

        self.evaluator = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, 16)
        )

    def forward(self, x):
        x = torch.reshape(x, (x.shape[0],1,8,8)) # można ten reshape dać gdzie indziej jak się znajdzie lepsze miejsce
        x = self.convblock1(x)
        x = self.convblock2(x)
        x = self.convblock3(x)
        x = self.convblock4(x)
        x = self.evaluator(x)
        return x

In [676]:
class Model3D(nn.Module):
    def __init__(self):
        super().__init__()
        self.convblock1 = nn.Sequential(
            nn.Conv3d(in_channels=1, out_channels=32, kernel_size=5, padding='same'),
            nn.ReLU(inplace=True),
            nn.BatchNorm3d(32),
            nn.AvgPool3d(2),
            nn.Dropout(p=0.05)
        )

        self.convblock2 = nn.Sequential(
            nn.Conv3d(in_channels=32, out_channels=64, kernel_size=5, padding='same'),
            nn.ReLU(inplace=True),
            nn.BatchNorm3d(64),
            nn.AvgPool3d(2),
            nn.Dropout(p=0.05)
        )

        self.convblock3 = nn.Sequential(
            nn.Conv3d(in_channels=64, out_channels=128, kernel_size=5, padding='same'),
            nn.ReLU(inplace=True),
            # nn.BatchNorm3d(128),
            nn.Dropout(p=0.05)
        )

        self.convblock4 = nn.Sequential(
            nn.Conv3d(in_channels=128, out_channels=256, kernel_size=5, padding='same'),
            nn.ReLU(inplace=True),
            # nn.BatchNorm3d(256),
            nn.Dropout(p=0.05)
        )

        self.evaluator = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, 16)
        )

    def forward(self, x):
        x = torch.reshape(x, (x.shape[0],1,4,4,4))
        x = self.convblock1(x)
        x = self.convblock2(x)
        x = self.convblock3(x)
        x = self.convblock4(x)
        x = self.evaluator(x)
        return x

### Batch loader

In [677]:
# Na bazie datasetu z labów - zwraca kolejne batche.
# Nie jest jakiś genialny, ale i tak go używam tylko tu do testów.
class Dataset:
  def __init__(self, data, labels, batch_size=1):
    self.data = data
    self.labels = labels
    self.batch_size = batch_size
    self.n_batches = len(self.data) // batch_size

  def __iter__(self):
    for i in range(self.n_batches):
      inputs = torch.zeros(self.batch_size, 1, 64) # shape zahardkodowany
      labels = torch.zeros(self.batch_size, self.labels.shape[1])

      for j in range(self.batch_size):
        inputs[j, 0] = self.data[i*self.batch_size+j]
        labels[j]= self.labels[i*self.batch_size+j]

      yield inputs, labels
  # todo - jakiś shuffle by się przydał, ale to tylko jeśli byśmy z tego korzystały

### Funkcje do treningu i testowania

In [678]:
# Funkcja przeprowadzająca traning

def train(net, loader, optimizer, criterion, epochs):
  for epoch in range(epochs):
    epoch_loss = 0.
    for i, (inputs, labels) in enumerate(loader):

      # zero the parameter gradients
      optimizer.zero_grad()

      outputs = net(inputs)
      loss = criterion(outputs, labels)

      loss.backward()
      optimizer.step()

      epoch_loss += loss
    if epoch % 5 == 4: # wypisywanie co 5 epok
      print(f'[{epoch + 1}, {i + 1:5d}] eloss: {epoch_loss:.3f}')

In [693]:
def test(network, test_set, n_classes = 16):
  network.eval() # Przełącza sieć w tryb testowania, m. in. wyłącza dropouta.
  err_matrix = torch.zeros((n_classes, n_classes), dtype=int)

  with torch.no_grad():

    l = 0 # Obliczam koszt, żeby wiedzieć, co się dzieje, docelowo tego nie będzie.
    for inputs, labels in test_set:

      outputs = network(inputs)
      l += criterion(outputs, labels)

      _, label = torch.max(labels, 1)
      _, predicted = torch.max(outputs, 1)

      for truth, prediction in zip(label, predicted):
        err_matrix[truth, prediction] += 1

  network.train() # Przełącza z powrotem w tryb treningu.
  return err_matrix, l

In [680]:
# Funkcja tworzy losowe etykiety one-hot.
def rand_and_predict(dataset_size, output_length):
  labels = torch.zeros(dataset_size, output_length)
  for i in range(dataset_size):
    hot = torch.randint(output_length,(1,))
    labels[i, hot] = 1
  return labels

## Test na losowych danych

In [717]:
inputs = torch.rand(256,1,64)
labels = rand_and_predict(256,16)

In [722]:
loader = Dataset(inputs, labels, batch_size=32)
criterion = nn.CrossEntropyLoss()
criterion_test = nn.CrossEntropyLoss()

### Model 1D

In [723]:
network1 = Model1D()
optimizer1 = optim.Adam(network1.parameters(), lr=0.001)

In [724]:
train(network1, loader, optimizer1, criterion, 10)

[5,     8] eloss: 0.968
[10,     8] eloss: 0.038


In [725]:
result1, loss = test(network1, loader)
print(f"Accuracy: {sum(sum(result1*torch.eye(16)))*100/sum(sum(result1)):.2f}%")
print(loss)
print(result1)

Accuracy: 100.00%
tensor(0.0329)
tensor([[ 9,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0, 21,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0, 16,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0, 20,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0, 17,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0, 20,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0, 13,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0, 13,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0, 21,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0, 20,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 13,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  9,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  9,  0,  0,  0],
     

### Model 2D

In [726]:
network2 = Model2D()
optimizer2 = optim.Adam(network2.parameters(), lr=0.001)

In [727]:
train(network2, loader, optimizer2, criterion, 20)

[5,     8] eloss: 15.003
[10,     8] eloss: 1.362
[15,     8] eloss: 0.934
[20,     8] eloss: 0.649


In [728]:
result2, loss = test(network2, Dataset(inputs, labels))
print(f"Accuracy: {sum(sum(result2*torch.eye(16)))*100/sum(sum(result2)):.2f}%")
print(loss)
print(result2)

Accuracy: 98.44%
tensor(14.8871)
tensor([[ 9,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0, 21,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0, 16,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0, 20,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0, 15,  0,  0,  0,  0,  0,  0,  0,  0,  1,  1,  0],
        [ 0,  0,  0,  0,  0, 20,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0, 13,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0, 13,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0, 21,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0, 20,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 13,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  9,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  9,  0,  0,  0],
     

### Model 3D

In [729]:
network3 = Model3D()
optimizer3 = optim.Adam(network3.parameters(), lr=0.001)

In [730]:
train(network3, loader, optimizer3, criterion, 30)

[5,     8] eloss: 20.926
[10,     8] eloss: 18.663
[15,     8] eloss: 15.994
[20,     8] eloss: 12.420
[25,     8] eloss: 9.254
[30,     8] eloss: 6.468


In [731]:
result3, loss = test(network3, Dataset(inputs, labels))
print(f"Accuracy: {sum(sum(result3*torch.eye(16)))*100/sum(sum(result3)):.2f}%")
print(loss)
print(result3)

Accuracy: 68.75%
tensor(215.7659)
tensor([[ 1,  0,  0,  0,  0,  0,  2,  0,  0,  0,  6,  0,  0,  0,  0,  0],
        [ 0, 18,  0,  0,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0],
        [ 0,  0, 14,  0,  0,  0,  0,  0,  0,  0,  1,  1,  0,  0,  0,  0],
        [ 0,  2,  0, 16,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0, 10,  0,  0,  1,  0,  0,  1,  0,  0,  5,  0,  0],
        [ 0,  0,  0,  0,  2, 16,  0,  1,  1,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0, 11,  0,  0,  0,  2,  0,  0,  0,  0,  0],
        [ 0,  0,  1,  0,  1,  2,  1,  8,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  1,  0,  0,  2,  1,  0,  1, 12,  0,  0,  0,  0,  4,  0,  0],
        [ 1,  2,  1,  0,  1,  1,  0,  1,  0, 10,  1,  0,  0,  0,  1,  1],
        [ 0,  0,  1,  0,  1,  0,  3,  0,  0,  0,  8,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  8,  0,  0,  0,  0,  1,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  4,  0,  0,  0,  5,  0,  0,  0,  0,  0],
    

## Notatki, żeby nie szukać ciągle w artykule, bałagan trochę, ale są

koszt - cross entropy

aktywacja lineara - relu

aktywacja po 16 neuronach - softmax

zrobili L1, L2 i dropout, ale nie wchodziłam w to dokładnie

batch - 64, 128 dawały najlepsze wyniki

eksperymentalnie sprawdzili, że 100 epok daje zbieżność

w adamie dali 0.0001 learning rate
